In [1]:
# ========================
# RIASEC FORM 42 PERNYATAAN (Bahasa Indonesia dari PDF)
# ========================

questions = [
    "Saya suka memperbaiki mobil.",
    "Saya suka mengerjakan teka-teki.",
    "Saya pandai menggunakan alat atau mesin.",
    "Saya suka bekerja dalam tim.",
    "Saya adalah orang yang ambisius, saya berusaha menjadi yang terbaik.",
    "Saya suka mengatur segala sesuatu (file, meja kerja).",
    "Saya suka bekerja di luar ruangan.",
    "Saya suka menyelidiki sesuatu.",
    "Saya pandai membuat karya seni.",
    "Saya suka membantu orang lain.",
    "Saya suka menjual sesuatu atau memengaruhi orang lain.",
    "Saya menyukai pekerjaan yang membutuhkan perhatian pada detail.",
    "Saya tidak keberatan bekerja dengan tangan saya.",
    "Saya suka mencari tahu bagaimana sesuatu bekerja.",
    "Saya suka tampil di depan orang (bernyanyi, menari, berbicara).",
    "Saya peduli dengan orang lain.",
    "Saya menyukai kompetisi.",
    "Saya pandai mengisi dokumen atau mengikuti prosedur.",
    "Saya suka bekerja dengan hewan.",
    "Saya suka melakukan eksperimen ilmiah.",
    "Saya suka menulis cerita atau puisi.",
    "Saya suka menjadi sukarelawan di masyarakat.",
    "Saya suka memimpin.",
    "Saya senang bekerja dengan angka atau data.",
    "Saya suka menggunakan alat berkebun atau alat pertukangan.",
    "Saya suka menganalisis masalah.",
    "Saya suka menggambar atau melukis.",
    "Saya suka bekerja dengan anak-anak.",
    "Saya suka meyakinkan orang untuk menerima ide saya.",
    "Saya menyukai tugas yang membutuhkan akurasi.",
    "Saya menikmati kegiatan fisik.",
    "Saya suka memecahkan masalah atau pertanyaan ilmiah.",
    "Saya suka menghadiri acara budaya atau seni.",
    "Saya senang mendengarkan masalah orang lain.",
    "Saya suka mengambil risiko.",
    "Saya suka bekerja dengan sistem organisasi.",
    "Saya suka bekerja di pertanian atau bidang alam.",
    "Saya suka membaca buku ilmiah atau teknis.",
    "Saya suka kegiatan seperti fotografi, menulis, atau desain grafis.",
    "Saya senang mengajari atau melatih orang lain.",
    "Saya suka memulai proyek saya sendiri.",
    "Saya suka bekerja dengan data dan angka secara sistematis."
]

riasec_types = ['Realistic', 'Investigative', 'Artistic', 'Social', 'Enterprising', 'Conventional']
riasec_index = {
    'Realistic':     [0, 6, 12, 18, 24, 30, 36],
    'Investigative': [1, 7, 13, 19, 25, 31, 37],
    'Artistic':      [2, 8, 14, 20, 26, 32, 38],
    'Social':        [3, 9, 15, 21, 27, 33, 39],
    'Enterprising':  [4, 10, 16, 22, 28, 34, 40],
    'Conventional':  [5, 11, 17, 23, 29, 35, 41]
}

riasec_scores = {k: 0 for k in riasec_types}

print("\nSilakan isi kuesioner berikut dari 1 (sangat tidak setuju) sampai 5 (sangat setuju):\n")

for i, q in enumerate(questions):
    while True:
        try:
            ans = int(input(f"{i+1}. {q} [1-5]: "))
            if 1 <= ans <= 5:
                for dim, idxs in riasec_index.items():
                    if i in idxs:
                        riasec_scores[dim] += ans
                break
            else:
                print("Masukkan harus antara 1 dan 5.")
        except:
            print("Input tidak valid. Masukkan angka 1 sampai 5.")

normalized_scores = [round(riasec_scores[k]/7, 2) for k in riasec_types]
print("\nSkor RIASEC Anda (rata-rata):")
for k, v in zip(riasec_types, normalized_scores):
    print(f"{k}: {v}")

print("\nArray skor untuk input model:")
print(normalized_scores)

user_riasec_array = normalized_scores



Silakan isi kuesioner berikut dari 1 (sangat tidak setuju) sampai 5 (sangat setuju):


Skor RIASEC Anda (rata-rata):
Realistic: 3.0
Investigative: 4.43
Artistic: 4.0
Social: 4.71
Enterprising: 4.14
Conventional: 4.43

Array skor untuk input model:
[3.0, 4.43, 4.0, 4.71, 4.14, 4.43]


In [2]:
import tensorflow as tf
import joblib
import numpy as np
import pandas as pd

# ========================
# 1. Load Model dan Scaler
# ========================
embedding_model = tf.keras.models.load_model("embedding_model.h5")
scaler = joblib.load("scaler.pkl")

# Load data pekerjaan
riasec_df = pd.read_csv("Dataset/interests_riasec_dataset.csv")
df_pivot = riasec_df[['O*NET-SOC Code', 'Title', 'Element Name', 'Data Value']] \
    .pivot_table(index=['O*NET-SOC Code', 'Title'], columns='Element Name', values='Data Value') \
    .reset_index()

riasec_types = ['Realistic', 'Investigative', 'Artistic', 'Social', 'Enterprising', 'Conventional']
df_pivot = df_pivot[['O*NET-SOC Code', 'Title'] + riasec_types]
riasec_scaled = scaler.transform(df_pivot[riasec_types])

# ========================
# 2. Fungsi Rekomendasi
# ========================
def recommend_jobs(user_riasec_scores, top_n=5):
    sample_user_df = pd.DataFrame([user_riasec_scores], columns=riasec_types)
    user_scores_scaled = scaler.transform(sample_user_df)
    user_embedding = embedding_model.predict(user_scores_scaled)
    job_embeddings = embedding_model.predict(riasec_scaled)

    user_norm = np.linalg.norm(user_embedding, axis=1, keepdims=True)
    job_norm = np.linalg.norm(job_embeddings, axis=1, keepdims=True)
    similarities = np.dot(job_embeddings, user_embedding.T) / (job_norm * user_norm.T + 1e-8)
    similarities = similarities.flatten()

    top_indices = similarities.argsort()[-top_n:][::-1]
    recommendations = df_pivot.iloc[top_indices][['O*NET-SOC Code', 'Title']].copy()
    recommendations['Similarity Score'] = similarities[top_indices]
    top_riasec = sample_user_df.iloc[0].sort_values(ascending=False).head(2).index.tolist()
    reason = f"Karena skor RIASEC tertinggi Anda adalah pada dimensi {top_riasec[0]} dan {top_riasec[1]}"
    recommendations['Alasan Rekomendasi'] = reason
    return recommendations

# ========================
# 3. Gunakan skor dari form PDF (misal dari variabel user_riasec_array)
# ========================
# Contoh saja, sesuaikan dari output notebook sebelumnya
user_riasec_array = [3.0, 4.43, 4.0, 4.71, 4.14, 4.43]

# Jalankan rekomendasi
hasil_rekomendasi = recommend_jobs(user_riasec_array, top_n=5)
print(hasil_rekomendasi)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 118ms/step
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step  
Element Name O*NET-SOC Code                                    Title  \
227              19-3032.00  Industrial-Organizational Psychologists   
232              19-3041.00                             Sociologists   
237              19-3094.00                     Political Scientists   
330              25-4012.00                                 Curators   
335              25-9031.00               Instructional Coordinators   

Element Name  Similarity Score  \
227                   0.959054   
232                   0.938510   
237                   0.935772   
330                   0.923312   
335                   0.913176   

Element Name                                 Alasan Rekomendasi  
227           Karena skor RIASEC tertinggi Anda adalah pada ...  
232           Karena skor RIASEC tertinggi Anda adalah pada ...  
237           Karena skor RIASEC tertinggi Anda adalah pada ...  
330           Kare